In [1]:
import pandas as pd
from datetime import datetime
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests, bs4

In [2]:
df = pd.read_csv('Stock_Trading_Data.txt', encoding='UTF-8')

In [3]:
df['trade_amount_max'] = df['trade_amount_max'].apply(lambda x: "${:,.2f}".format(x))
df['trade_amount_min'] = df['trade_amount_min'].apply(lambda x: "${:,.2f}".format(x))

In [4]:
df = df.dropna(subset = ["ticker"])

In [5]:
df = df[df.ticker != "--"]

In [6]:
df["min_max"] = df["trade_amount_min"] + "-" + df["trade_amount_max"]

In [7]:
for i in range(len(df)):
    day = df['trade_date'][i][:2]
    month = df['trade_date'][i][2:5]
    year = df['trade_date'][i][5:]
    date_full = month + "/" + day + "/" + year
    date_format = "%b/%d/%Y"
    df['trade_date'][i] = datetime.strptime(date_full, date_format)

/var/folders/q2/5v2xmgf12v118bg1jrms5ph80000gn/T/ipykernel_19117/1889992851.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['trade_date'][i] = datetime.strptime(date_full, date_format)


KeyError: 44

In [8]:
for i in range(len(df)):
    if df.iloc[i,1] == 0:
        df.iloc[i,1] = "Republican"
    elif df.iloc[i,1] == 1.0:
        df.iloc[i,1] = "Democrat"
    elif df.iloc[i,1] == 2.0:
        df.iloc[i,1] = "Independent"

In [9]:
for i in range(len(df)):
    if df.iloc[i,2] == 1:
        df.iloc[i,2] = "Male"
    elif df.iloc[i,2] == 0:
        df.iloc[i,2] = "Female"

In [10]:
df["Sector"] = ""

In [11]:
df = df[[
    "senator",
    "party",
    "sex",
    "trade_date",
    "owner",
    "ticker",
    "Sector",
    "type",
    "min_max",
    "one_month_sell_compare",
    "one_month_sell",
    "one_mo_vtsax_change",
    "buy_compare_one_year",
    "buy_one_year_change",
    "one_year_vtsax_change"
]]

In [12]:
df = df.rename(columns={
    "senator":"Senator",
    "party":"Party",
    "sex":"Sex",
    "trade_date":"Date of Trade",
    "owner":"Owner",
    "ticker":"Ticker",
    "type":"Transaction Type",
    "min_max":"Range of Transaction Value",
    "one_month_sell_compare":"Stock Value vs Market (1 month)",
    "one_month_sell":"Stock Value Change (1 month)",
    "one_mo_vtsax_change":"Market Change (1 month)",
    "buy_compare_one_year":"Stock Value vs Market (1 year)",
    "buy_one_year_change":"Stock Value Change (1 year)",
    "one_year_vtsax_change":"Market Change (1 year)"
})

In [13]:
unique_tickers = df["Ticker"].unique()

In [14]:
sector_string = []
for ticker in unique_tickers:
    url = f"https://finance.yahoo.com/quote/{ticker}/profile?p={ticker}"
    headers = {"User-Agent":"Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    results = soup.find_all('p', class_='D(ib) Va(t)')
    if results == []:
        sector_string.append("No sector")
    for result in results:
        sector = result.find("span", class_="Fw(600)").text
        sector_string.append(sector)

In [15]:
ticker_sector_dict = {unique_tickers[i]: sector_string[i] for i in range(len(unique_tickers))}

In [16]:
ticker_list = df["Ticker"].tolist()
sector_list = []
for i in range(len(ticker_list)):
    for key in ticker_sector_dict:
        if ticker_list[i] == key:
            sector_list.append(ticker_sector_dict[key])
df["Sector"] = sector_list
df.head()

,Senator,Party,Sex,Date of Trade,Owner,Ticker,Sector,Transaction Type,Range of Transaction Value,Stock Value vs Market (1 month),Stock Value Change (1 month),Market Change (1 month),Stock Value vs Market (1 year),Stock Value Change (1 year),Market Change (1 year)
0,Thomas Carper,Democrat,Male,2021-01-29 00:00:00,Spouse,XCAPX,No sector,Purchase,"$15,000.00-$50,000.00",NaN,NaN,0.031875,NaN,NaN,NaN
1,Patrick Toomey,Republican,Male,2021-01-28 00:00:00,Child,GME,Consumer Cyclical,Sale,"$1,000.00-$15,000.00",-0.454702,-0.438378,0.016324,NaN,NaN,NaN
2,Jerry Moran,Republican,Male,2021-01-27 00:00:00,Spouse,COST,Consumer Defensive,Sale,"$1,000.00-$15,000.00",-0.098641,-0.044025,0.054616,NaN,NaN,NaN
3,Jerry Moran,Republican,Male,2021-01-27 00:00:00,Self,COST,Consumer Defensive,Sale,"$1,000.00-$15,000.00",-0.098641,-0.044025,0.054616,NaN,NaN,NaN
4,Patrick Toomey,Republican,Male,2021-01-27 00:00:00,Child,SHOP,Technology,Sale,"$1,000.00-$15,000.00",0.136826,0.191442,0.054616,NaN,NaN,NaN
